In [1]:
# Import necessary libraries
import h2o
from h2o.automl import H2OAutoML

# Initialize the H2O cluster
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.20.1" 2023-08-24; OpenJDK Runtime Environment (build 11.0.20.1+1-post-Ubuntu-0ubuntu120.04); OpenJDK 64-Bit Server VM (build 11.0.20.1+1-post-Ubuntu-0ubuntu120.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpvt5x0h4x
  JVM stdout: /tmp/tmpvt5x0h4x/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpvt5x0h4x/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.3
H2O_cluster_version_age:,1 month and 18 days
H2O_cluster_name:,H2O_from_python_unknownUser_301keu
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.500 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [2]:
# Load your data into a H2OFrame
data = h2o.import_file('/kaggle/input/optiver-trading-at-the-close/train.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [5]:
# Specify the response and predictor columns
response_column = 'target'
predictor_columns = data.columns
predictor_columns.remove("target")

In [6]:
# Need to detect best model on reduced dataset due to size
## Determine the ratio for your sample
ratio = 0.0001
## Split frame
splits = data.split_frame(ratios = [ratio], seed = 1234)
# Sample is the first split
sample = splits[0]
print("sample size train+val is:",sample.nrows)

# Split the sample into training and validation sets
train_sample, val_sample = sample.split_frame(ratios=[0.8], seed=1)

# Initialize and train the H2OAutoML model
#aml = H2OAutoML(max_models=20, seed=1)
aml = H2OAutoML(include_algos=["DeepLearning"])
aml.train(x=predictor_columns, y=response_column, training_frame=train_sample)

# View the AutoML Leaderboard
lb = aml.leaderboard
print(lb.head(rows=lb.nrows))

# Predict on the validation set
preds = aml.predict(val_sample)

## Convert the predictions back to a pandas dataframe (optional)
#preds_df = h2o.as_list(preds)

sample size train+val is: 525
AutoML progress: |
15:37:44.570: _train param, Dropping bad and constant columns: [row_id]

███████████████████████████████████████████████████████████████| (done) 100%
model_id                                                   rmse       mse      mae    rmsle    mean_residual_deviance
DeepLearning_grid_2_AutoML_1_20231011_153744_model_10   9.03397   81.6127  6.13581      nan                   81.6127
DeepLearning_grid_2_AutoML_1_20231011_153744_model_3    9.03598   81.649   6.1378       nan                   81.649
DeepLearning_grid_2_AutoML_1_20231011_153744_model_5    9.05906   82.0665  6.13896      nan                   82.0665
DeepLearning_grid_3_AutoML_1_20231011_153744_model_5    9.07753   82.4016  6.17727      nan                   82.4016
DeepLearning_grid_2_AutoML_1_20231011_153744_model_11   9.08251   82.4921  6.10823      nan                   82.4921
DeepLearning_grid_3_AutoML_1_20231011_153744_model_14   9.088     82.5917  6.16933      nan   

In [7]:
# Extract best model
best_model = aml.leader

# Save the model
h2o.save_model(model=best_model, path="/kaggle/working/", force=True)

'/kaggle/working/DeepLearning_grid_2_AutoML_1_20231011_153744_model_10'

In [8]:
# Split the data into training and validation sets
train, val = data.split_frame(ratios=[0.8], seed=1)

In [9]:
# Continue training the model on the full data set
best_model.train(x=predictor_columns, y=response_column, training_frame=train)

deeplearning Model Build progress: |

/opt/conda/lib/python3.10/site-packages/h2o/estimators/estimator_base.py:192: RuntimeWarning: Dropping bad and constant columns: [row_id]
  warnings.warn(mesg["message"], RuntimeWarning)


█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1697038529279_7


Status of Neuron Layers: predicting target, regression, gaussian distribution, Quadratic loss, 43,601 weights/biases, 521.9 KB, 1,593,681 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight            weight_rms            mean_bias             bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ---------------------  ----------  ---------------------  --------------------  --------------------  -----------------------
    1        15       Input      0.0
    2        200      Rectifier  0.0        0.0   0.0   0.04264777874605109    0.10577505826950073    0.0         0.01797607714612968    0.22406494617462158   0.24353269238941944   0.14440786838531494
    3        200      Rectifier  0.0        0.0   0.0   0.11610351652145764    0.11470359563827515    0.0         -0.035569753455646995  0.07961350679397583   0.8915318422561194    0.036612704396247864
    4        1        Linear                0.0   0.0   0.0009416696192238305  0.0005472067277878523  0.0         -0.004822426676181522  0.038797587156295776  -0.06138430568646466  1.0971281125650402e-154

ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 85.80861173910142
RMSE: 9.263293784561808
MAE: 6.25034908563895
RMSLE: NaN
Mean Residual Deviance: 85.80861173910142

Scoring History: 
    timestamp            duration          training_speed    epochs     iterations    samples      training_rmse    training_deviance    training_mae    training_r2
--  -------------------  ----------------  ----------------  ---------  ------------  -----------  ---------------  -------------------  --------------  -------------
    2023-10-11 16:34:45  0.000 sec                           0          0             0            nan              nan                  nan             nan
    2023-10-11 16:34:53  8.655 sec         6374 obs/sec      0.0108044  1             45279        9.36206          87.6481              6.34234         0.00570222
    2023-10-11 16:35:00  16.494 sec        9287 obs/sec      0.032579   3             136532       9.32179          86.8958              6.29728         0.0142365
    2023-10-11 16:35:07  23.641 sec        10571 obs/sec     0.0542944  5             227537       9.3146           86.7618              6.28398         0.0157568
    2023-10-11 16:35:14  30.043 sec        11506 obs/sec     0.0759628  7             318345       9.31086          86.6921              6.29013         0.0165471
    2023-10-11 16:35:20  36.577 sec        12074 obs/sec     0.097788   9             409810       9.31114          86.6972              6.29309         0.016489
    2023-10-11 16:35:26  42.584 sec        12629 obs/sec     0.119585   11            501155       9.28297          86.1735              6.25735         0.022431
    2023-10-11 16:35:33  48.729 sec        12987 obs/sec     0.141277   13            592063       9.29557          86.4076              6.28373         0.0197754
    2023-10-11 16:35:41  56.668 sec        12817 obs/sec     0.162933   15            682819       9.28133          86.1431              6.26566         0.0227754
    2023-10-11 16:35:46  1 min  2.413 sec  13180 obs/sec     0.184825   17            774563       9.2767           86.0572              6.2577          0.0237495
    2023-10-11 16:35:53  1 min  9.003 sec  13307 obs/sec     0.20657    19            865693       9.30403          86.5649              6.30321         0.01799
    2023-10-11 16:35:59  1 min 15.097 sec  13494 obs/sec     0.228324   21            956862       9.26849          85.905               6.25209         0.0254767
    2023-10-11 16:36:05  1 min 20.698 sec  13733 obs/sec     0.249933   23            1.04742e+06  9.27231          85.9758              6.25395         0.0246734
    2023-1

In [10]:
# Predict on the validation set
preds = best_model.predict(val)

deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%


In [11]:
# Save the model
h2o.save_model(model=best_model, path="/kaggle/working/", force=True)

'/kaggle/working/DeepLearning_model_python_1697038529279_7'